In [1]:
import langchain
import openai
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain import OpenAI, VectorDBQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import DirectoryLoader
import magic
import nltk
import textwrap

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/manojkr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
#loader = UnstructuredFileLoader('/Users/manojkr/Documents/Development/esg/sustainability-report.pdf', mode="elements")
loader = DirectoryLoader('./bofa/', glob="**/*.txt")


documents = loader.load()

print (f'You have {len(documents)} document(s) in your data')

documents[:5]

You have 1 document(s) in your data


[Document(page_content='A NNUA L REPORT 2022\n\n˜°°˛˜˝ ˆˇ˘\x17ˆ\x16 \x15\x14\x15\x15\n\nTable of\n\nContents\n\n14\n\nEight Lines of\n\nBusiness\n\nOur eight lines of\n\nbusiness\n\n14\n\nRetail Banking\n\n16\n\nPreferred Banking\n\n18\n\nMerrill\n\n20\n\nPrivate Bank\n\n22\n\nBusiness Banking\n\n24\n\nGlobal Commercial\n\nBanking\n\n26\n\nGlobal Corporate &\n\nInvestment Banking\n\n28\n\nGlobal Markets\n\n30\n\n02\n\nCompany\n\nPerformance\n\nA letter from Chair\n\nand CEO Brian\n\nMoynihan\n\n02\n\nLetter from Lead\n\nIndependent Director\n\n11\n\nBoard of Directors\n\nand Executive\n\nManagement Team\n\n12\n\n32\n\nLocal & Digital\n\nSpotlights\n\nLocal Markets\n\nOrganization\n\n32\n\nDigital capabilities\n\n34\n\n36\n\nCommunity Impact\n\nDiverse entrepreneurs\n\n36\n\nSustainable fnance\n\n38\n\nJobs initiatives\n\n39\n\nOther highlights\n\n40\n\n42\n\nHuman Capital\n\nManagement Update\n\nA letter from\n\nSheri Bronstein\n\n42\n\nBenefts overview\n\n44\n\nTalent & development\n\

In [22]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=300)

texts = text_splitter.split_documents(documents)

texts[:10]

[Document(page_content='A NNUA L REPORT 2022\n\n˜°°˛˜˝ ˆˇ˘\x17ˆ\x16 \x15\x14\x15\x15\n\nTable of\n\nContents\n\n14\n\nEight Lines of\n\nBusiness\n\nOur eight lines of\n\nbusiness\n\n14\n\nRetail Banking\n\n16\n\nPreferred Banking\n\n18\n\nMerrill\n\n20\n\nPrivate Bank\n\n22\n\nBusiness Banking\n\n24\n\nGlobal Commercial\n\nBanking\n\n26\n\nGlobal Corporate &\n\nInvestment Banking\n\n28\n\nGlobal Markets\n\n30\n\n02\n\nCompany\n\nPerformance\n\nA letter from Chair\n\nand CEO Brian\n\nMoynihan\n\n02\n\nLetter from Lead\n\nIndependent Director\n\n11\n\nBoard of Directors\n\nand Executive\n\nManagement Team\n\n12\n\n32\n\nLocal & Digital\n\nSpotlights\n\nLocal Markets\n\nOrganization\n\n32\n\nDigital capabilities\n\n34\n\n36\n\nCommunity Impact\n\nDiverse entrepreneurs\n\n36\n\nSustainable fnance\n\n38\n\nJobs initiatives\n\n39\n\nOther highlights\n\n40\n\n42\n\nHuman Capital\n\nManagement Update\n\nA letter from\n\nSheri Bronstein\n\n42\n\nBenefts overview\n\n44\n\nTalent & development\n\

In [23]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

docsearch = Chroma.from_documents(texts, embeddings)

#qa = VectorDBQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", vectorstore=docsearch)
qa = VectorDBQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


In [55]:
query="which year is this report for?"
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 This report is for 2022.


[Document(page_content='Review for our total Assets Under Management. Additionally, refer\n\nto the same table and the Client Balances subsection for further\n\ndetail on the Corporation’s client balances under management.\n\n2022 Environmental Data will be published in Q2 2023.\n\n68 | BANK OF AMERICA 2022\n\nThis page intentionally left blank.\n\nBANK OF AMERICA 2022\n\n| 69\n\nmiscellaneous & appendix\n\nCautionary Information and\n\nForward-Looking Statements\n\nThis report, including the Stakeholder Capitalism Metrics section,\n\ncontains certain statements regarding Responsible Growth and\n\nenvironmental, social and governance information and opinions,\n\nincluding metrics, aspirations, targets, goals, commitments,\n\ncumulative values and sustainability strategy (collectively, the\n\nSustainability Information). The Sustainability Information\n\nincluded in this report may consider disclosure recommendations\n\nand broader definitions of materiality used by certain voluntary\n\

In [61]:
query = "What is Bank of America's core principle?"
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
#print('\n'.join(textwrap.wrap(result['source_documents'], width=150, replace_whitespace=False)))
result['source_documents']

 Bank of America's core principle is Responsible Growth, which is focused on helping transmit the economy by lending, investing, and working alongside
their clients to help them succeed, while also supporting their teammates and local communities.


[Document(page_content='or paying people fairly and equitably. At Bank of America, we look\n\nat it in this straightforward way: How can we rely on and help\n\nthe innovation and energy of capitalism to address the priorities\n\nof our communities and society AND beneft our shareholders?\n\nCapitalism provides the money, the creativity, and the expertise\n\nto solve the needs of society. We enable our customers to drive\n\ncapitalism. It is what we do.\n\n$27.5B\n\nnet income\n\n$3T\n\nbalance sheet\n\nWe once again delivered both\n\nprofts and purpose — the genius\n\nof the AND as we call it — and we\n\ndid it by delivering on the tenets of\n\nResponsible Growth.\n\nThroughout this report we provide details about how we do this:\n\nHow we help transmit the economy by lending, investing, and\n\nworking alongside our clients to help them succeed. About all we\n\ndo to support our teammates and be a Great Place to Work for\n\nthem. The many ways we partner in local markets to help suppor

In [33]:
query="What are the tenets of responsible growth?"
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 The tenets of responsible growth are: (1) win in the market, no excuses; (2) grow with a customer-focused strategy; (3) grow with the right risk
principles; and (4) grow in a sustainable manner: be the best place to work, share our success with our communities and drive operational excellence.


[Document(page_content='Responsible growth has four straightforward tenets, including “we\n\nmust grow in a sustainable manner.” Bank of America focuses on\n\nresponsible growth by: (1) sharing its success with the communities\n\nit serves, including through its wide-ranging ESG activities; (2) being\n\na great place to work for its teammates; and (3) driving operational\n\nexcellence so that it can continue to invest in its employees and\n\ncapabilities.\n\nData security\n\nDescription of approach to identifying and addressing data\n\nsecurity risks\n\nRefer to our Information Security and Privacy Policies available on the\n\nGovernance page of our corporate website and the Compliance and\n\nOperational Risk Management section on pages 127-128 of the 2022\n\nFinancial Review for detail on our approach to data security risks.\n\nSASB: FN-CB-230a.2; FN-CF-230a.3\n\nSystemic risk management\n\nGlobal Systemically Important Bank (G-SIB) score, by category\n\nOur G-SIB surcharge is 2.5%. R

In [36]:
query="""
The tenets of responsible growth are: (1) win in the market, no excuses; (2) grow with a customer-focused strategy; (3) grow with the right risk
principles; and (4) grow in a sustainable manner: be the best place to work, share our success with our communities and drive operational excellence.

How did Bank do with respect to the 2nd Tenet?
"""
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 Bank of America focused on the 2nd tenet of responsible growth by acquiring new clients and deepening relationships with existing ones. They also
tracked how well they delivered integrated capabilities by the number of client referrals between their lines of business in their local markets. They
saw more than 7.5 million total referrals among their operating units. They also focused on reducing clients' reliance on overdrafts and providing
alternatives to help clients manage their deposit accounts and avoid fees, which led to record-high levels of customer satisfaction for Bank of
America in 2022.


[Document(page_content='crisis ensued and your company performed well once\n\nagain, even as we met our clients’ borrowing demands,\n\nbuilt reserves, traded through uncertain markets, and\n\nhelped our employees, clients and our communities deal\n\nwith the pandemic and its afermath. Again, Responsible\n\nGrowth served us well.\n\nDuring 2022, the U.S. and the world continued to face\n\ncivil tensions from the pandemic and lockdowns, social\n\nand racial conficts, a war in Europe and energy supply\n\ndisruptions, and infation.\n\nResponsible Growth\n\n1. Win in the market, no excuses.\n\n2. Grow with a customer-focused strategy.\n\n3. Grow with the right risk principles.\n\n2 | BANK OF AMERICA 2022\n\n4. Grow in a sustainable manner: Be the best\n\nplace to work, share our success with our\n\ncommunities and drive Operational Excellence.\n\nThe strength of our balance sheet, risk management, organic\n\ngrowth, and expense management became a catalyst that drove\n\nour fnancial perform

In [46]:
query="""
How many new checking accounts were open in 2022?
"""
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 One million new checking accounts were opened in 2022.


[Document(page_content='In 2022, we saw more than\n\n7.5 million total referrals among\n\nour operating units.\n\nIn 2022, we helped consumer clients to open a record one million\n\nnet new checking accounts. We also opened 465,000 Consumer\n\n6 | BANK OF AMERICA 2022\n\nInvestment accounts through our online investment platform,\n\nMerrill Edge, resulting in $28 billion in net client fows. And we\n\nadded more than a million new consumer credit card accounts\n\nin three of the four quarters in 2022.\n\nPreferred Rewards — our fagship loyalty program that\n\nrecognizes and rewards clients for doing business with us across\n\nall products — recorded another strong year of growth, with\n\n10.2 million total clients enrolled. This represents a 9% year-overyear increase, with the program retaining its best-in-class 99%\n\nannualized membership retention rate.\n\nSmall business owners benefted from Bank of America’s products,\n\nscale and expertise at our retail fnancial centers across the 

In [49]:
query="what are the lines of businesses? list them all.  How many are there?"
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 The lines of business are Consumer Banking (includes Retail and Preferred Banking), Global Wealth & Investment Management (includes Merrill and The
Private Bank), Global Banking (includes Business Banking, Commercial Banking and Global Corporate & Investment Banking) and Global Markets. There are
8 lines of business.


[Document(page_content='NOTE 23 Business Segment Information\n\nThe Corporation reports its results of operations through the\n\nfollowing four business segments: Consumer Banking, Global\n\nWealth & Investment Management, Global Banking and Global\n\nMarkets, with the remaining operations recorded in All Other.\n\nConsumer Banking\n\nConsumer Banking offers a diversified range of credit, banking\n\nand investment products and services to consumers and small\n\nbusinesses. Consumer Banking product offerings include\n\ntraditional savings accounts, money market savings accounts,\n\nCDs and IRAs, checking accounts, and investment accounts and\n\nproducts, as well as credit and debit cards, residential\n\nmortgages and home equity loans, and direct and indirect loans\n\nto consumers and small businesses in the U.S. Consumer\n\nBanking includes the impact of servicing residential mortgages\n\nand home equity loans.\n\nGlobal Wealth & Investment Management\n\nGWIM provides a high-touch clie

In [53]:
query="Who leads the retail banking"
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 Holly O'Neill


[Document(page_content='reporting to Dean Athanasia, president of Regional Banking.\n\nGLOBAL MARKETS\n\nProvides services across the world’s debt, equity, commodity\n\nand foreign exchange markets. This includes liquidity,\n\nhedging strategies, industry-leading insights, analytics and\n\ncompetitive pricing for corporate, commercial, government\n\nand institutional investor clients.\n\nThis business is led by Jim DeMare.\n\nBANK OF AMERICA 2022\n\n| 15\n\nretail banking\n\neight lines of business\n\nHelping our clients thrive\n\ntoday and in the future\n\nIn Retail Banking, we support the fnancial health\n\nof nearly 68 million U.S. consumer clients through\n\nour fnancial center locations, ATMs and awardwinning digital banking capabilities, including Online\n\nand Mobile Banking platforms, Erica® and Zelle®.\n\nUnder the leadership of Holly O’Neill, our Retail Banking teammates provide a\n\npersonalized experience that supports clients on their fnancial journey — as they budget,', l

In [54]:
query="Who leads Global Markets?"
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 Jim DeMare


[Document(page_content='estate firms, not-for-profit companies, large global corporations,\n\nfinancial institutions, leasing clients, and mid-sized U.S.-based\n\nbusinesses requiring customized and integrated financial advice\n\nand solutions.\n\nGlobal Markets\n\nGlobal Markets offers sales and trading services and research\n\nservices to institutional clients across fixed-income, credit,\n\ncurrency, commodity and equity businesses. Global Markets\n\nprovides market-making, financing, securities clearing,\n\nsettlement and custody services globally to institutional investor\n\nclients in support of their investing and trading activities. Global\n\nMarkets product coverage includes securities and derivative\n\nproducts in both the primary and secondary markets. Global\n\nMarkets also works with commercial and corporate clients to\n\nprovide risk management products. As a result of marketmaking activities, Global Markets may be required to manage\n\nrisk in a broad range of financial 

In [57]:
query="Tell me about Erica?"
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 Erica is our virtual financial assistant that makes it easier for consumers to manage day-to-day finances. It has assisted 33.5 million total users
with nearly 1.2 billion interactions since its launch.


[Document(page_content='means that our products and services must evolve in line with\n\nour clients’ unique fnancial needs. Consumer customers continue\n\nto tell us they value our digital capabilities. In 2022, your company:\n\nAdded three million active digital banking users, improving upon\n\na record performance in 2021\n\nHad 56 million verifed users with 44 million active and engaged\n\nin the past 90 days\n\nExperienced 11.6 billion digital logins\n\nGrew digital sales 22%, representing nearly 50% of overall\n\nconsumer sales\n\nErica®, our artifcial intelligence (AI)-based virtual fnancial assistant,\n\nalso continues to attract increased client engagement as it has\n\nproven to be a critical channel to help us deliver important information\n\nto clients and address their questions and concerns in real-time. Erica\n\nhandled roughly 145 million interactions in the fourth quarter and\n\nrecently passed a billion interactions since its introduction.\n\nAmong the most signifcant 

In [59]:
query="What is bank's commitment to the environment?"
result = qa({'query': query})

print('\n'.join(textwrap.wrap(result['result'], width=150, replace_whitespace=False)))
result['source_documents']

 Bank of America has a commitment to environmental sustainability and has invested in Economic and Social Progress and Environmental Sustainability.
They have $45.5 billion of client balances in Sustainable & Impact Investments which integrate environmental, social, and governance (ESG) factors.
They also have a Environmental and Social Risk Policy Framework which outlines their approach to incorporating ESG factors in credit analysis,
investment banking and brokerage activities, and wealth management processes and strategies. They also provide guidance to clients on how to navigate
the transition to net zero greenhouse gas emissions.


[Document(page_content='our investment in Economic and Social Progress and Environmental\n\nSustainability.\n\nSASB: FN-IB-410a.2\n\nAmount of assets under management, by asset class, that\n\nemploy (1) integration of environmental, social, and governance\n\n(ESG) issues, (2) sustainability themed investing, and (3)\n\nscreening\n\nAs of December 31, 2022, Bank of America had $45.5 billion of client\n\nbalances in Sustainable & Impact Investments which we define as\n\ninvestment strategies that intentionally integrate environmental,\n\nsocial or governance factors in order to avoid harm by reducing\n\nnegative social or environmental effects or managing risk by limiting\n\ncertain exposures, benefit stakeholders by supporting positive social\n\nor environmental practices, or contribute to solutions that address\n\nspecific and measurable sustainability outcomes or impact.\n\nSASB FN-AC-410a.1\n\n62 | BANK OF AMERIC A 2022\n\nTHEME\n\nMETRIC/QUESTION\n\nRESPONSE/FRAMEWORK ALIGNMENT\n\nC